# WSI - Zadanie 4 <br/>
### 4.12.2023 <br/>
Wojciech Pobocha
318399


In [175]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Funkcje służące do przygotowania danych do trenowania i testowania modelu

In [176]:
def splitData(data, ratio):
    dataLength = len(data)
    numberOfTrain = int(dataLength * ratio)
    test = data.iloc[numberOfTrain:dataLength]
    train = data.iloc[0:numberOfTrain]
 
    return train, test

In [177]:
def getRepeatedValues(column):
    values = []
    for i in range(0, len(column)):
        if column[i] not in values:
            values.append(column[i])
    return values

In [178]:
def oneHotEncoding(data, columnNames):
    valuesDictionary = {}
    for columnName in columnNames:
        column = data[columnName]
        repeatedValues = getRepeatedValues(column)
        valuesDictionary[columnName] = repeatedValues
        for i in range(0, len(repeatedValues)):
            data[columnName] = data[columnName].replace(repeatedValues[i], i)
    return data, valuesDictionary

In [179]:
def decodeOneHotEncoding(data, decodingDictionary):
    for column in decodingDictionary:
        for i in range(0, len(decodingDictionary[column])):
            data[column] = data[column].replace(i, decodingDictionary[column][i])
    return data

In [180]:
def labelDataset(data, columnNames, threshold):
    data['passed exam'] = -1
    for i in range(0, len(data)):
        for column in columnNames:
            if data[column][i] <= threshold:
                data['passed exam'][i] = 1
                break
    data.drop(columnNames, axis=1, inplace=True)
    return data


# Definicja funkcji jądrowych

In [181]:
def rbfKernel(u, v, sigma=1):
    return np.exp(-np.linalg.norm(u-v)**2/(2*sigma**2))

In [182]:
def linearKernel(u, v):
    return np.dot(u, v)

# Definicja funkcji decyzyjnej
Parametry funkcji:
- `x` - argument funkcji,
- `parameters` - wektor parametrów (alpha1,...,alphaN,b), dłuższe od X o 1, by uwzględnić parametr b, został wybrany na podstawie minimalizacji funkcji celu
- `X` -  wektor argumentów modelu, pochodzący z danych treningowych
- `Y` - wektor oczekiwanych wartości module, pochodzący z danych treningowych
- `kernel` - funkcja jądrowa

In [183]:
def f(x, parameters, b, X, Y, kernel):
    sum = 0
    for i in range(len(X)):
        sum += parameters[i] * Y[i] * kernel(X[i], x)
    sum += -b
    return sum

In [184]:
def wNormSquare(X, Y, parameters, kernel):
    sum = 0
    for i in range(len(X)):
        for j in range(len(X)):
            sum += parameters[i] * parameters[j] * Y[i] * Y[j] * kernel(X[i], X[j])
    return sum

# Funkcja celu

In [185]:
def functionToMinimize(parameters, X, Y, kernel, C):
    sum = 0
    for i in range(len(X)):
        sum += max(1-f(X[i],parameters,parameters[-1],X,Y,kernel)*Y[i], 0)
    sum += C * wNormSquare(X,Y,parameters,kernel)
    return sum
        

# Gradient funkcji celu

Wzór gradientu wyprowadzony analitycznie

In [186]:
def gradientFunctionToMinimize(parameters, X, Y, kernel, C):
    gradient = np.zeros(len(X)+1)
    for n in range(len(X)):
        condition = 1-f(X[n],parameters,parameters[-1],X,Y,kernel)*Y[n]
        firstSum = 0
        secondSum = 0
        for i in range(len(X)):
            if condition > 0:
                firstSum += (-parameters[n]*Y[n]*kernel(X[n], X[i]))*Y[i]
            secondSum += parameters[n]*Y[n]*Y[i]*kernel(X[n], X[i]) + parameters[i]* Y[i]*Y[n]*kernel(X[i], X[n])
        gradient[n] = firstSum + C * secondSum
        if condition > 0:
            gradient[-1] += Y[n]
    return gradient

# Funkcja służąca do minimalizacji parametrów (alpha1,...alphaN,b)

Skorzystałem z algorytmu najmniejszego spadku implementowanego na pierwszym laboratorium

In [187]:
def fastestDescent(alfa, maxIterations, startingPoint, gradient,functionToMinimize):
    valuesVector = np.array([startingPoint])
    dimension = valuesVector.shape[1]
    iterations = 1
    bestValue = np.inf
    gradientVector = gradient(valuesVector[0])
    while iterations < maxIterations:
        valuesVector = np.vstack((valuesVector,np.zeros(dimension)))
        currentValue = functionToMinimize(valuesVector[iterations-1])
        if bestValue > currentValue:
            bestParameters = valuesVector[iterations-1]
            bestValue = currentValue
        valuesVector[iterations] = valuesVector[iterations-1] - alfa * gradientVector
        
        gradientVector = gradient(valuesVector[iterations])
        iterations +=1

    return bestParameters, valuesVector


In [188]:
def svmGradientAdapter(X, Y, kernel, C):
    def svmGradient(parameters):
        return gradientFunctionToMinimize(parameters, X, Y, kernel, C)
    return svmGradient

In [189]:
def svmFunctionToMinimizeAdapter(X, Y, kernel, C):
    def svmFunctionToMinimize(parameters):
        return functionToMinimize(parameters, X, Y, kernel, C)
    return svmFunctionToMinimize

In [190]:
def getExamsDataSets(ratio=0.1):
    data = pd.read_csv('data/exams.csv')
    columnsToEncode = ['gender','race/ethnicity',	'parental level of education',	'lunch',	'test preparation course']
    data, decodingDictionary = oneHotEncoding(data,columnsToEncode)
    data = labelDataset(data, ['math score', 'reading score', 'writing score'], 60)
    train, test = splitData(data, ratio)
    return train, test

In [191]:
def vectorizeData(data):
    X = np.vstack([data['gender'],data['race/ethnicity'],data['parental level of education'],data['lunch'],data['test preparation course']]).T
    Y = data['passed exam'].to_numpy()
    return X, Y

In [192]:
def fitSVM(X,Y,kernel,C):
    startingParameters = np.random.uniform(-10,10,len(X)+1)
    parameters, valuesVector = fastestDescent(0.01, 1000, startingParameters, svmGradientAdapter(X, Y, kernel, C), svmFunctionToMinimizeAdapter(X, Y, kernel, C))
    return parameters

# Definicja metryk służących do oceny modelu

In [193]:

def getConfusionMatrix(testSetXs, testSetYs,trainSetX,trainSetY, kernel, parameters, b):
    confusionMatrix = np.zeros((2,2))
    for i in range(len(testSetXs)):
        val = f(testSetXs[i],parameters,b,trainSetX,trainSetY,kernel)
        if val < 0:
            if testSetYs[i] > 0:
                confusionMatrix[0][1] += 1
            else:
                confusionMatrix[0][0] += 1
        else:
            if testSetYs[i] < 0:
                confusionMatrix[1][0] += 1
            else:
                confusionMatrix[1][1] += 1
    return confusionMatrix


In [194]:
def getAccuracy(confusionMatrix):
    return (confusionMatrix[0][0] + confusionMatrix[1][1]) / np.sum(confusionMatrix)

In [195]:
def getPrecision(confusionMatrix):
    return confusionMatrix[0][0] / (confusionMatrix[0][0] + confusionMatrix[0][1])

In [196]:
def getRecall(confusionMatrix):
    return confusionMatrix[0][0] / (confusionMatrix[0][0] + confusionMatrix[1][0])

# RBF kernel

In [197]:
train, test = getExamsDataSets(0.1)
X, Y = vectorizeData(train)
XTest, YTest = vectorizeData(test)
kernel = rbfKernel
C=1
parameters = fitSVM(X,Y,kernel,C)

In [198]:
matrix = getConfusionMatrix(XTest, YTest, X, Y, kernel, parameters, parameters[-1])
print(matrix)
print(getAccuracy(matrix))
print(getPrecision(matrix))
print(getRecall(matrix))


[[348. 276.]
 [170. 106.]]
0.5044444444444445
0.5576923076923077
0.6718146718146718


# Komentarz do wyników
Niestety wyniki są niezadowalające, prawdopodobnie popełniłem błąd w implementacji gradientu funkcji, jednak nie mogłem go znaleźć. Oprócz tego, testowanie algorytmu jest bardzo wolne, dlatego zbiór trenujący jest taki mały w porównaniu do testującego.

# Linear kernel

In [199]:
train, test = getExamsDataSets(0.1)
X, Y = vectorizeData(train)
XTest, YTest = vectorizeData(test)
kernel = linearKernel
C=1
parameters = fitSVM(X,Y,kernel,C)

/tmp/ipykernel_144501/206866904.py:5: RuntimeWarning: overflow encountered in scalar add
  sum += parameters[i] * parameters[j] * Y[i] * Y[j] * kernel(X[i], X[j])
/tmp/ipykernel_144501/206866904.py:5: RuntimeWarning: overflow encountered in scalar multiply
  sum += parameters[i] * parameters[j] * Y[i] * Y[j] * kernel(X[i], X[j])
/tmp/ipykernel_144501/206866904.py:5: RuntimeWarning: invalid value encountered in scalar multiply
  sum += parameters[i] * parameters[j] * Y[i] * Y[j] * kernel(X[i], X[j])
/tmp/ipykernel_144501/882997021.py:4: RuntimeWarning: overflow encountered in scalar add
  sum += max(1-f(X[i],parameters,parameters[-1],X,Y,kernel)*Y[i], 0)
/tmp/ipykernel_144501/236129591.py:10: RuntimeWarning: overflow encountered in scalar add
  secondSum += parameters[n]*Y[n]*Y[i]*kernel(X[n], X[i]) + parameters[i]* Y[i]*Y[n]*kernel(X[i], X[n])
/tmp/ipykernel_144501/2821799041.py:4: RuntimeWarning: invalid value encountered in scalar multiply
  sum += parameters[i] * Y[i] * kernel(X[i],

In [202]:
matrix = getConfusionMatrix(XTest, YTest, X, Y, kernel, parameters, parameters[-1])
print(matrix)
print(getAccuracy(matrix))
# print(getPrecision(matrix))
# print(getRecall(matrix))


[[  0.   0.]
 [518. 382.]]
0.42444444444444446


# Podsumowanie algorytmu i eksperymentów
